In [2]:
import pandas as pd
from nba_api.stats.endpoints import playercareerstats, commonplayerinfo, playerprofilev2
from nba_api.stats.static import players
from nba_api.stats.static import teams

In [4]:
df = pd.read_csv('all_players_v2.csv')

In [5]:
nba_players = [i for i in players.get_players() if i['is_active']==True]
teams_df = pd.DataFrame(teams.get_teams())
players_df = pd.DataFrame(nba_players)

In [68]:
latest_df.head()

,PLAYER_ID,SEASON_ID,LEAGUE_ID,TEAM_ID,TEAM_ABBREVIATION,PLAYER_AGE,GP,GS,MIN,FGM,...,REB,AST,STL,BLK,TOV,PF,PTS,AST_per_game,REB_per_game,PTS_per_game
7,1627826,2021-22,0,1610612746,LAC,25.0,76,76,1852,310,...,644,120,36,77,114,203,785,1.578947,8.473684,10.328947
16,203469,2021-22,0,1610612757,POR,29.0,27,0,355,51,...,125,22,8,6,19,56,140,0.814815,4.629630,5.185185
17,1630209,2021-22,0,1610612748,MIA,24.0,56,12,706,130,...,294,49,17,20,41,84,299,0.875000,5.250000,5.339286
21,1629027,2021-22,0,1610612737,ATL,23.0,76,76,2652,711,...,284,737,72,7,303,128,2155,9.697368,3.736842,28.355263
38,201152,2021-22,0,1610612759,SAS,33.0,26,1,370,74,...,93,59,23,7,30,38,158,2.269231,3.576923,6.076923


In [79]:
ad_id = 203076
df_per_game = df.groupby(['PLAYER_ID','SEASON_ID','TEAM_ID'], as_index=False).agg({'PTS':'max',
                                                                                  'REB':'max',
                                                                                   'AST':'max',
                                                                                  'GP':'max',
                                                                                  'PLAYER_AGE':'max'})
# we divide the points scored by the games in season, as we have the player's contribution
# for every game in the simulation.
games_in_season = 82
df_per_game['PTS_PER_GAME'] = df_per_game['PTS']/games_in_season
df_per_game['REB_PER_GAME'] = df_per_game['REB']/games_in_season
df_per_game['AST_PER_GAME'] = df_per_game['AST']/games_in_season
df_per_game[df_per_game.PLAYER_ID==dwhite_id].tail()

,PLAYER_ID,SEASON_ID,TEAM_ID,PTS,REB,AST,GP,PLAYER_AGE,PTS_PER_GAME,REB_PER_GAME,AST_PER_GAME
2620,1628401,2019-20,1610612759,765,223,241,68,25.0,9.329268,2.719512,2.939024
2621,1628401,2020-21,1610612759,553,107,127,36,26.0,6.743902,1.304878,1.548780
2622,1628401,2021-22,0,992,261,366,75,27.0,12.097561,3.182927,4.463415
2623,1628401,2021-22,1610612738,286,88,92,26,27.0,3.487805,1.073171,1.121951
2624,1628401,2021-22,1610612759,706,173,274,49,27.0,8.609756,2.109756,3.341463


In [80]:
dwhite_id = 1628401
grouped = df_per_game.groupby(by=['PLAYER_ID','SEASON_ID','TEAM_ID'], as_index=False).agg({'PTS_PER_GAME':'mean',
                                             'AST_PER_GAME':'mean',
                                            'REB_PER_GAME':'mean',
                                            'PLAYER_AGE':'mean',
                                                      'GP':'sum'}).sort_values(
                        by=['SEASON_ID','TEAM_ID','GP'], ascending=[False,False, False])\
                    .drop_duplicates(subset=['PLAYER_ID','SEASON_ID'], keep='first')
grouped[grouped.PLAYER_ID==dwhite_id].head(10)

,PLAYER_ID,SEASON_ID,TEAM_ID,PTS_PER_GAME,AST_PER_GAME,REB_PER_GAME,PLAYER_AGE,GP
2624,1628401,2021-22,1610612759,8.609756,3.341463,2.109756,27.0,49
2621,1628401,2020-21,1610612759,6.743902,1.548780,1.304878,26.0,36
2620,1628401,2019-20,1610612759,9.329268,2.939024,2.719512,25.0,68
2619,1628401,2018-19,1610612759,8.085366,3.207317,3.012195,24.0,67
2618,1628401,2017-18,1610612759,0.658537,0.109756,0.317073,23.0,17


In [81]:
grouped = grouped.merge(players_df[['id','full_name']], left_on='PLAYER_ID',right_on='id')
grouped = grouped.merge(teams_df[['id','full_name']], left_on='TEAM_ID', right_on='id')
grouped.drop(columns=['id_x','id_y'], inplace=True)
grouped.rename(columns={'full_name_x':'full_name_player', 'full_name_y':'full_name_team'}, inplace=True)

In [82]:
#grouped[grouped.PLAYER_ID==ariza_id].head(10)
grouped.GP.max()

82

In [83]:
eligible_seasons = ['2017-18','2018-19','2019-20','2020-21','2021-22']
clean_grouped = grouped[grouped.SEASON_ID.isin(eligible_seasons)]
clean_grouped.SEASON_ID.unique()

array(['2021-22', '2020-21', '2019-20', '2018-19', '2017-18'],
      dtype=object)

In [84]:
clean_grouped.to_csv('../db/initial_migration/players_2016_until_2022.csv', index=False)

In [85]:
ariza_id=2772
clean_grouped[clean_grouped.PLAYER_ID==ariza_id]

,PLAYER_ID,SEASON_ID,TEAM_ID,PTS_PER_GAME,AST_PER_GAME,REB_PER_GAME,PLAYER_AGE,GP,full_name_player,full_name_team
363,2772,2018-19,1610612764,7.378049,2.012195,2.756098,33.0,43,Trevor Ariza,Washington Wizards
1509,2772,2021-22,1610612747,1.170732,0.329268,1.000000,36.0,24,Trevor Ariza,Los Angeles Lakers
1668,2772,2017-18,1610612745,9.536585,1.280488,3.585366,33.0,67,Trevor Ariza,Houston Rockets
1790,2772,2019-20,1610612758,2.341463,0.609756,1.780488,35.0,32,Trevor Ariza,Sacramento Kings
2284,2772,2020-21,1610612748,3.439024,0.670732,1.756098,36.0,30,Trevor Ariza,Miami Heat
